In [ ]:
!mkdir DATA
!unzip -qq {'/content/drive/MyDrive/데이콘/반도체/open.zip'} -d /content/DATA

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.ensemble import IsolationForest
from tqdm import tqdm
import random

In [ ]:
# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(1004) # Seed 고정

In [ ]:
# 데이터 로딩 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): csv 파일의 경로.
            transform (callable, optional): 샘플에 적용될 Optional transform.
        """
        self.df = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df['img_path'].iloc[idx]
        image = Image.open('/content/DATA'+img_path[1:])
        if self.transform:
            image = self.transform(image)
        target = torch.tensor([0.]).float()
        return image, target

# 이미지 전처리 및 임베딩
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_data = CustomDataset(csv_file='/content/DATA/train.csv', transform=transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=False)

In [ ]:
model = models.wide_resnet50_2(pretrained=True)
model.fc = nn.Linear(in_features=2048, out_features=1, bias=True)
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scheduler = CosineAnnealingLR(optimizer, T_max=100, eta_min=0.00001)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Wide_ResNet50_2_Weights.IMAGENET1K_V1`. You can also use `weights=Wide_ResNet50_2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
def train(model, train_loader, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total = 0

        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels.view(-1, 1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            predictions = (torch.sigmoid(outputs) > 0.5).float()
            running_corrects += torch.sum(predictions == labels.view(-1, 1)).item()
            total += labels.size(0)

        scheduler.step()

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = running_corrects / total

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')


In [ ]:
# 모델 학습 실행
train(model, train_loader, criterion, optimizer, scheduler, num_epochs=5)

Epoch 1/10, Loss: 0.0029, Accuracy: 1.0000
Epoch 2/10, Loss: 0.0004, Accuracy: 1.0000
Epoch 3/10, Loss: 0.0001, Accuracy: 1.0000
Epoch 4/10, Loss: 0.0001, Accuracy: 1.0000
Epoch 5/10, Loss: 0.0001, Accuracy: 1.0000


KeyboardInterrupt: 

In [ ]:
# 사전 학습된 모델 로드
model.eval()  # 추론 모드로 설정

# 특성 추출을 위한 모델의 마지막 레이어 수정
model = torch.nn.Sequential(*(list(model.children())[:-1]))

model.to(device)

# 이미지를 임베딩 벡터로 변환
def get_embeddings(dataloader, model):
    embeddings = []
    with torch.no_grad():
        for images, _ in tqdm(dataloader):
            images = images.to(device)
            emb = model(images)
            embeddings.append(emb.cpu().numpy().squeeze())
    return np.concatenate(embeddings, axis=0)

train_embeddings = get_embeddings(train_loader, model)

100%|██████████| 7/7 [00:25<00:00,  3.65s/it]


In [ ]:
train_embeddings

array([[0.58294976, 0.51278746, 0.33975375, ..., 0.32456827, 0.17273414,
        0.18773916],
       [0.5654894 , 0.5268161 , 0.3140535 , ..., 0.32336867, 0.14982766,
        0.18977794],
       [0.5547641 , 0.5397621 , 0.3074204 , ..., 0.33938074, 0.16519792,
        0.19574451],
       ...,
       [0.56647867, 0.47445217, 0.36659226, ..., 0.32170135, 0.17426944,
        0.19460368],
       [0.5497859 , 0.53079414, 0.34334657, ..., 0.31717616, 0.15633187,
        0.19284533],
       [0.59784967, 0.4498407 , 0.2785041 , ..., 0.37942493, 0.15373312,
        0.2144058 ]], dtype=float32)

In [ ]:
train_embeddings.shape

(213, 2048)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

# Isolation Forest 모델 학습
clf = IsolationForest(random_state=77)
clf.fit(train_embeddings)

IsolationForest(random_state=77)

In [ ]:
# 테스트 데이터에 대해 이상 탐지 수행
test_data = CustomDataset(csv_file='/content/DATA/test.csv', transform=transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

test_embeddings = get_embeddings(test_loader, model)
test_pred = clf.predict(test_embeddings)

# Isolation Forest의 예측 결과(이상 = -1, 정상 = 1)를 이상 = 1, 정상 = 0으로 변환
test_pred = np.where(test_pred == -1, 1, 0)

100%|██████████| 4/4 [00:11<00:00,  2.85s/it]


In [ ]:
submit = pd.read_csv('/content/DATA/sample_submission.csv')
submit['label'] = test_pred
submit.to_csv('submission.csv', index=False)

In [ ]:
submit.head()

,id,label
0,TEST_000,0
1,TEST_001,0
2,TEST_002,0
3,TEST_003,0
4,TEST_004,0


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

# Isolation Forest 모델 학습
clf = IsolationForest(random_state=42)
clf.fit(train_embeddings)

IsolationForest(random_state=42)

In [ ]:
# 테스트 데이터에 대해 이상 탐지 수행
test_data = CustomDataset(csv_file='/content/DATA/test.csv', transform=transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

test_embeddings = get_embeddings(test_loader, model)
test_pred = clf.predict(test_embeddings)

# Isolation Forest의 예측 결과(이상 = -1, 정상 = 1)를 이상 = 1, 정상 = 0으로 변환
test_pred = np.where(test_pred == -1, 1, 0)

100%|██████████| 4/4 [00:01<00:00,  3.90it/s]


In [ ]:
test_embeddings.shape

(100, 2048)

In [ ]:
submit = pd.read_csv('/content/DATA/sample_submission.csv')
submit['label'] = test_pred
submit.to_csv('submission.csv', index=False)

# Recon

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers


def l2_distance(z):
    diff = tf.expand_dims(z, axis=1) - tf.expand_dims(z, axis=0)
    return tf.reduce_sum(diff ** 2, axis=-1)


class PairwiseSimilarity(layers.Layer):
    def __init__(self, sigma=1.0):
        super(PairwiseSimilarity, self).__init__()
        self.sigma = sigma

    def call(self, z):
        return tf.exp(-l2_distance(z) / self.sigma)


class ContextualSimilarity(layers.Layer):
    def __init__(self, k):
        super(ContextualSimilarity, self).__init__()
        self.k = k

    def call(self, z):
        distances = l2_distance(z)
        kth_nearst = -tf.math.top_k(-distances, k=self.k, sorted=True)[0][:, -1]
        mask = tf.cast(distances <= tf.expand_dims(kth_nearst, axis=-1), tf.float32)

        similarity = tf.matmul(mask, mask, transpose_b=True) / tf.reduce_sum(mask, axis=-1, keepdims=True)
        R = mask * tf.transpose(mask)
        similarity = tf.matmul(similarity, R, transpose_b=True) / tf.reduce_sum(R, axis=-1, keepdims=True)
        return 0.5 * (similarity + tf.transpose(similarity))


class ReConPatch(keras.Model):
    def __init__(self, input_dim, embedding_dim, projection_dim, alpha, margin=0.1):
        super(ReConPatch, self).__init__()
        self.alpha = alpha
        self.margin = margin

        # embedding & projection layers
        self.embedding = layers.Dense(embedding_dim)
        self.projection = layers.Dense(projection_dim)

        # ema ver of embedding & projection layers
        self.ema_embedding = layers.Dense(embedding_dim, trainable=False)
        self.ema_projection = layers.Dense(projection_dim, trainable=False)

        # initialize layers
        self.embedding.build((None, input_dim))
        self.projection.build((None, embedding_dim))
        self.ema_embedding.build((None, input_dim))
        self.ema_projection.build((None, embedding_dim))

        # ema operator
        self.ema = tf.train.ExponentialMovingAverage(decay=0.9)
        self.update_ema()

        self.pairwise_similarity = PairwiseSimilarity(sigma=1.0)
        self.contextual_similarity = ContextualSimilarity(k=3)

    def call(self, x):
        return self.embedding(x)

    def train_step(self, x):
        h_ema = self.ema_embedding(x)
        z_ema = self.ema_projection(h_ema)

        p_sim = self.pairwise_similarity(z_ema)
        c_sim = self.contextual_similarity(z_ema)
        w = self.alpha * p_sim + (1 - self.alpha) * c_sim

        with tf.GradientTape() as tape:
            h = self.embedding(x)
            z = self.projection(h)

            # Contrastive loss
            distances = tf.sqrt(l2_distance(z) + 1e-9)
            delta = distances / tf.reduce_mean(distances, axis=-1, keepdims=True)
            rc_loss = tf.reduce_sum(tf.reduce_mean(
                w * (delta ** 2) + (1 - w) * (tf.nn.relu(self.margin - delta) ** 2),
                axis=-1
            ))
        # Update weights
        self.optimizer.minimize(rc_loss, self.trainable_variables, tape=tape)
        # Update EMA
        self.update_ema()

        return {"rc_loss": rc_loss}

    def update_ema(self):
      ema_variables = self.embedding.variables + self.projection.variables
      for var in ema_variables:
          ema_var = self.ema.average(var)
          if ema_var is not None:
              var.assign(ema_var)

In [ ]:
model = ReConPatch(input_dim=2048, embedding_dim=256, projection_dim=128, alpha=0.5, margin=0.1)

# 옵티마이저 및 모델 컴파일
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer)

# 모델 훈련
model.fit(train_embeddings, epochs=1000)

Epoch 1/1000
7/7 [==============================] - 1s 4ms/step - rc_loss: 7.5287
Epoch 2/1000
7/7 [==============================] - 0s 4ms/step - rc_loss: 7.4970
Epoch 3/1000
7/7 [==============================] - 0s 4ms/step - rc_loss: 7.5982
Epoch 4/1000
7/7 [==============================] - 0s 4ms/step - rc_loss: 7.5164
Epoch 5/1000
7/7 [==============================] - 0s 4ms/step - rc_loss: 7.9783
Epoch 6/1000
7/7 [==============================] - 0s 4ms/step - rc_loss: 7.4002
Epoch 7/1000
7/7 [==============================] - 0s 4ms/step - rc_loss: 7.4108
Epoch 8/1000
7/7 [==============================] - 0s 4ms/step - rc_loss: 7.5043
Epoch 9/1000
7/7 [==============================] - 0s 4ms/step - rc_loss: 7.5653
Epoch 10/1000
7/7 [==============================] - 0s 4ms/step - rc_loss: 7.4333
Epoch 11/1000
7/7 [==============================] - 0s 4ms/step - rc_loss: 7.5647
Epoch 12/1000
7/7 [==============================] - 0s 4ms/step - rc_loss: 7.7994
Epoch 13/1000

In [ ]:
import tensorflow as tf

class ReConPatchModified(ReConPatch):
    def infer_step(self, x):
        """
        추론을 위한 메서드. 가중치 업데이트를 수행하지 않습니다.
        """
        # Exponential Moving Average (EMA)를 사용하여 임베딩을 계산합니다.
        h_ema = self.ema_embedding(x)
        z_ema = self.ema_projection(h_ema)

        # 입력 데이터에 대한 임베딩을 계산합니다.
        h = self.embedding(x)
        z = self.projection(h)

        # Pairwise 및 Contextual Similarity를 계산합니다.
        p_sim = self.pairwise_similarity(z)
        c_sim = self.contextual_similarity(z)
        w = self.alpha * p_sim + (1 - self.alpha) * c_sim

        # Contrastive loss를 계산합니다.
        distances = tf.sqrt(l2_distance(z) + 1e-9)
        delta = distances / tf.reduce_mean(distances, axis=-1, keepdims=True)
        rc_loss = tf.reduce_sum(tf.reduce_mean(
            w * (delta ** 2) + (1 - w) * (tf.nn.relu(self.margin - delta) ** 2),
            axis=-1
        ))

        # 이 메서드에서는 가중치 업데이트를 하지 않습니다.
        # 추론 결과인 rc_loss를 반환합니다.
        return rc_loss

    def predict(self, dataset):
        """
        입력 데이터셋에 대한 추론을 수행하고 rc_loss를 출력합니다.
        """
        rc_losses = []
        for batch in dataset:
            rc_loss = self.infer_step(batch)
            rc_losses.append(rc_loss.numpy())

        # 모든 배치에 대한 rc_loss를 반환합니다.
        return rc_losses


In [ ]:
import tensorflow as tf

# train_embedding 배열을 TensorFlow 텐서로 변환
train_embedding_tensor = tf.constant(train_embeddings)

# TensorFlow 데이터셋 생성
# ReConPatch 모델은 비지도 학습에 가깝기 때문에, 레이블이 필요 없습니다.
# 여기서는 입력 데이터를 그대로 레이블로 사용합니다.
train_dataset = tf.data.Dataset.from_tensor_slices((train_embedding_tensor, train_embedding_tensor))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(32)

In [ ]:
train_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 2048), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2048), dtype=tf.float32, name=None))>

In [ ]:
# ReConPatch 모델 인스턴스 생성
input_dim = train_embeddings.shape[1]  # 임베딩 차원
model = ReConPatch(input_dim=input_dim, embedding_dim=256, projection_dim=128, alpha=0.5, margin=0.1)

# 옵티마이저와 손실 함수 설정
model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 학습
model.fit(train_dataset, epochs=1000)

# 모델 평가 (여기서는 학습 데이터로 간단히 평가)
# 실제 사용 시 별도의 검증 데이터셋을 준비해야 함

Epoch 1/1000
7/7 [==============================] - 1s 5ms/step - rc_loss: 0.6844
Epoch 2/1000
7/7 [==============================] - 0s 5ms/step - rc_loss: 0.6465
Epoch 3/1000
7/7 [==============================] - 0s 4ms/step - rc_loss: 0.5462
Epoch 4/1000
7/7 [==============================] - 0s 5ms/step - rc_loss: 0.5677
Epoch 5/1000
7/7 [==============================] - 0s 5ms/step - rc_loss: 0.5610
Epoch 6/1000
7/7 [==============================] - 0s 4ms/step - rc_loss: 0.5365
Epoch 7/1000
7/7 [==============================] - 0s 5ms/step - rc_loss: 0.5355
Epoch 8/1000
7/7 [==============================] - 0s 5ms/step - rc_loss: 0.5415
Epoch 9/1000
7/7 [==============================] - 0s 5ms/step - rc_loss: 0.5449
Epoch 10/1000
7/7 [==============================] - 0s 5ms/step - rc_loss: 0.6147
Epoch 11/1000
7/7 [==============================] - 0s 5ms/step - rc_loss: 0.5017
Epoch 12/1000
7/7 [==============================] - 0s 5ms/step - rc_loss: 0.5148
Epoch 13/1000

In [ ]:
def predict(model, embeddings):
    # TensorFlow 텐서로 변환
    embeddings_tensor = tf.constant(embeddings)
    # 배치 처리를 위한 데이터셋 생성
    dataset = tf.data.Dataset.from_tensor_slices(embeddings_tensor)
    dataset = dataset.batch(32)

    predictions = []
    for batch in dataset:
        logits = model(batch, training=False)  # training=False로 설정하여 추론 모드 활성화
        predicted_classes = tf.argmax(logits, axis=1)
        predictions.extend(predicted_classes.numpy())

    return np.array(predictions)
# 예: test_embeddings를 사용하여 추론 수행
test_predictions = predict(model, test_embeddings)

# 예측 결과 출력
print("Test Predictions:", test_predictions)


Test Predictions: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [ ]:
# ReConPatch 모델 인스턴스화
model = ReConPatch(input_dim=512, embedding_dim=512, projection_dim=256, alpha=1, margin=0.1)
model.compile(optimizer='adam')

# 모델 학습
model.fit(train_dataset, epochs=10)

# 모델 평가 (여기서는 학습 데이터로 간단히 평가)
# 실제 사용 시 별도의 검증 데이터셋을 준비해야 함
print("모델 평가:")
model.evaluate(train_dataset)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

# 데이터 준비: 여기서는 예제를 위한 임시 데이터를 생성합니다.
# 실제 사용 시에는 적절한 입력 데이터를 준비해야 합니다.
input_dim = 512 # 예제 입력 차원
embedding_dim = 512  # 임베딩 차원
projection_dim = 32  # 프로젝션 차원

x_train = np.random.random((num_samples, input_dim)).astype(np.float32)

# 모델 인스턴스 생성
model = ReConPatch(input_dim, embedding_dim, projection_dim, alpha=0.5)

# 컴파일러 설정: 옵티마이저, 손실 함수 등
model.compile(optimizer=Adam(learning_rate=1e-3))

# 학습 과정: x_train을 입력과 목표 모두로 사용합니다.
# 이 모델은 자기 자신을 학습하는 self-supervised 모델의 특성을 가집니다.
model.fit(x_train, x_train, epochs=10, batch_size=32)